## Import modules

Make sure that Python 3 and the following modules (recommended version ID) are installed on your computer:

In [1]:
# to install with pip run:
# pip install -r requirements.txt

with open('requirements.txt', 'r') as f:
    print(''.join(f.readlines()))

jupyter==1.0.0
pyFirmata==1.1.0
numpy==1.18.4
opencv-python==4.2.0.34
sounddevice


In [2]:
# include controllers to the path
import sys, os
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'controllers'))

import cv2
import threading
import math
import time
import random
import json
import datetime
import os, shutil
import numpy as np
import multiprocessing as mp

# controllers
import nbimporter
from controllers.situtils import FPSTimes
from controllers.camera import WebcamStream
from controllers.video import VideoWriter
from controllers.position import PositionTracker
from controllers.sound import SoundController

## Load experiment settings

For every experimental cofiguration you can copy the original 'settings.json' file, build your own specific experimental preset, save it in this folder as e.g. 'settings_elena.json' and load it here instead of 'settings.json'.

In [3]:
cfg_filename = 'settings.json'

In [4]:
with open(cfg_filename) as json_file:
    cfg = json.load(json_file)
cfg['experiment']['experiment_date'] = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# print loaded settings
#print(json.dumps(cfg, indent=4))

## Initialize session folder

Run the upcoming cell, to create a session folder and to save the chosen experimetal parameters to a JSON-file ("experiment_id_parameters.json"). The session folder will be created here where this notebook is located.

In [5]:
# This session's protocols will be saved to this folder
cfg_exp = cfg['experiment']
experiment_id = "%s_%s_%s" % (cfg_exp['subject'], cfg_exp['experiment_type'], cfg_exp['experiment_date'])
save_to = os.path.join('sessions', experiment_id)
             
if not os.path.exists(save_to):
    os.makedirs(save_to)

# update paths (assuming this paths are relative to this notebook)
cfg['video']['file_path'] = os.path.join(save_to, 'video.avi')
cfg['position']['file_path'] = os.path.join(save_to, 'positions.csv')
cfg['experiment']['file_path'] = os.path.join(save_to, 'events.csv')
cfg['sound']['file_path'] = os.path.join(save_to, 'sounds.csv')
cfg['position']['background_file'] = os.path.join('assets', 'background.png')
    
# Saves all parameters to a JSON file with the user-defined "Experiment ID" as filename
with open(os.path.join(save_to, experiment_id + '_parameters.json'), 'w') as f:
    json.dump(cfg, f, indent=4)

In [6]:
class Island:    
    def __init__(self, x, y, radius, sound_id, is_distractor=False):
        self.x = x
        self.y = y
        self.r = radius
        self.sound_id = sound_id
        self.is_distractor = is_distractor

In [7]:
def generate_islands(arena_x, arena_y, arena_radius, island_radius, distractors=0):
    # TODO write this function
    return [Island(arena_x, arena_y, island_radius, 1)]  # list of Islands

## Start the experiment

This cell contains code for animal tracking. We hope that the comments provided in the code suffice to understand the individual steps and to adjust them to your own setup and needs, if necessary.

- press 's' to start recording
- press 's' again to stop recording
- press 'q' to quit

The experiment will stop automatically if the pre-defined session duration is reached.

In [16]:
# actual sound selector: 0 - silence, 1 - foraging, 2 - target, 3 - distractor
sound = mp.Value('i', 1)

# experiment status: 1 - idle, 2 - running (recording, logging), 0 - stopped
status = mp.Value('i', 1)

# first start the camera stream
vs = WebcamStream(cfg['camera'])
vs.start()

# init video recorder
vw = VideoWriter(status, vs, cfg['video'])
vw.start()

# start position tracking
pt = PositionTracker(status, vs, cfg['position'])
pt.start()

# playing sound in a separate process for performance
sc = mp.Process(target=SoundController.run, args=(sound, status, cfg['sound']))
sc.start()

fps = FPSTimes()
names = ['camera', 'video', 'position', 'main']
trial = 1
rewards = 0
t_start = None
trial_start = 0
phase = 1  # 1 - foraging, 2 - inter-trial interval
cfg_exp = cfg['experiment']
COLORS = {
    'red': (0,0,255), 'green': (127,255,0), 'blue': (255,127,0), 'yellow': (0,127,255), \
    'black': (0,0,0), 'white': (255,255,255)
}
islands = []

def timeout(t_start):
    return time.time() - t_start > cfg_exp['session_duration'] if t_start is not None else False

try:
    while trials <= cfg_exp['trial_number'] and not timeout(t_start):
        frame = vs.read()
        if frame is None:
            time.sleep(0.1)
            continue # wait for the stream
            
        fps.count()
        status_color = COLORS['green'] if status.value == 1 else COLORS['red']

        # -------- prepare the video frame ---------------
        
        # mask space outside arena
        frame = cv2.bitwise_and(src1=frame, src2=pt.mask)
        #frame = cv2.subtract(frame, pt.background)
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # draw position center and contours
        if pt.x is not None:
            cv2.circle(frame, (pt.x, pt.y), 10, status_color, -1)
            cv2.drawContours(frame, [pt.contour], 0, status_color, 1, cv2.LINE_AA)            

        # add FPS indicators
        for i, ctrl in enumerate([vs, vw, pt, fps]):
            cv2.putText(frame, '%s: %.2f FPS' % (names[i], ctrl.get_avg_fps()), 
                     (10, 30 + 20*(i+1)), cv2.FONT_HERSHEY_DUPLEX, .5, COLORS['white'])

        # size of the arena and status indicator
        cv2.circle(frame, (cfg_exp['arena_x'], cfg_exp['arena_y']), cfg_exp['arena_radius'], COLORS['red'], 2)
        cv2.circle(frame, (20,20), 10, status_color, -6)

        # draw islands
        if len(islands) > 0:
            for island in islands:
                clr = COLORS['red'] if island.is_distractor else COLORS['green']
                cv2.circle(frame, (island.x, island.y), island.r, clr, 2)

        # stopwatch
        stopwatch = 'Time: %.2f' % float(time.time() - t_start) if t_start is not None else 'Time: Idle'
        cv2.putText(frame, stopwatch, (10, 590), cv2.FONT_HERSHEY_DUPLEX, .5, COLORS['white'])

        # trial countdown (TODO add island countdown)
        if status.value > 1:
            text = 'Trial: %.2f' % float(cfg_exp['trial_duration'] - (time.time() - trial_start))
            cv2.putText(frame, text, (10, 610), cv2.FONT_HERSHEY_DUPLEX, .5, COLORS['white'])
            cv2.putText(frame, 'Trial: %s' % trial, (10, 630), cv2.FONT_HERSHEY_DUPLEX, .5, COLORS['white'])
            
            # rewards
            cv2.putText(frame, 'Rewards: %s' % rewards, (10, 650), cv2.FONT_HERSHEY_DUPLEX, .5, COLORS['white'])
            
        cv2.imshow('Press (s)-to start/stop, (q)-to end', frame)

        # -------- experiment logic ---------------
        
        c_time = time.time()
        
        # animals is either foraging (phase == 1) or in the inter-trial-interval (phase == 2)
        if phase == 1: # foraging
            if c_time - trial_start > cfg_exp['trial_duration']:  # trial failed
                next_trial_start = c_time + 5 + 10 * np.random.rand()  # random b/w 5-15 sec
                sound.value = 4  # punishment
                islands = []
                phase = 2
                trials += 1
                
            else:
                # check if animal in the island and for how long
                target = [i for i in islands if not i.is_distractor][0]
                distractors = [i for i in islands if  i.is_distractor]
                
                if (pt.x - target.x)**2 + (pt.y - target.y)**2 <= cfg_exp['target_radius']:
                    if target_since is None:  # just entered target
                        target_since = c_time
                        sound.value = 2
                    elif c_time - target_since > cfg_exp['target_duration']: # successful trial
                        next_trial_start = c_time + 5 + 10 * np.random.rand()  # random b/w 5-15 sec
                        sound.value = 0  # silence
                        islands = []
                        phase = 2
                        trials += 1
                        rewards += 1
                
                else:
                    target_since = None
                    in_distractor = False
                    
                    for isl in distractors:  # maybe animal is in one of the distractors
                        if (pt.x - isl.x)**2 + (pt.y - isl.y)**2 <= cfg_exp['target_radius']:
                            sound.value = isl.sound_id
                            in_distractor = True
                    
                    if not in_distractor:  # outside of the islands
                        sound.value = 1
            
        elif phase == 2:  # inter-trial-interval
            if c_time > next_trial_start:
                # init_new_trial
                islands = generate_islands(cfg_exp['arena_y'], cfg_exp['arena_y'], cfg_exp['arena_radius'], cfg_exp['target_radius'])
                sound.value = 1
                phase = 1
        
        # -------- key press events ---------------
        
        k = cv2.waitKey(33)
        if k == ord('q'):
            break

        if k == ord('s'):
            if status.value == 1: # start the session
                t_start = time.time()
                status.value = 2
                phase = 1
                islands = generate_islands(cfg_exp['arena_y'], cfg_exp['arena_y'], cfg_exp['arena_radius'], cfg_exp['target_radius'])
                trial_start = t_start
                
            elif status.value == 2:  # pause the session
                status.value = 1
                islands = []
            
        if k == ord('a'):
            sound.value = 0 if sound.value == 1 else 1

finally:
    status.value = 0
    time.sleep(0.01)
    cv2.destroyAllWindows()
    sc.join()
    for ctrl in [pt, vw, vs]:
        ctrl.stop()

Webcam stream 960.0:720.0 at 20.00 FPS started


ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred


Sound stopped
Position tracker stopped
Video writer stopped
Camera released
